In [1]:
# Necessary Packages
import os
import numpy as np
import cv2
import imghdr
import tensorflow as tf
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Input

C:\Users\acer\AppData\Local\Temp\ipykernel_17872\3873863118.py:4: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


In [2]:
# Dataset Source: https://www.kaggle.com/datasets/mohamedchahed/dog-breeds
# Folder is loaded and saved as a directory "dog-breeds"
r_state = 27
tf.random.set_seed(r_state)
np.random.seed(r_state)

In [3]:
extensions = ['png','jpg','jpeg']
file_path = os.path.join(os.curdir,r'dog-breeds') 

assert os.path.exists(file_path) # check that the folder exists

In [78]:
def remove_some_img(folder): # removes corrupted / poor images (just in case)
    for f in os.listdir(folder):
        for i in os.listdir(os.path.join(folder,f)):
            ip = os.path.join(folder,f,i)
            try:
                img = cv2.imread(ip) # try to see if image can be read by openCV
                ext = imghdr.what(ip) # check for extension
                if ext not in extensions: 
                    # check if img ext is valid (based on what is in the extensions list)
                    print(ip)
                    os.remove(ip)
            except Exception as e:
                print(ip)
                os.remove(ip)

remove_some_img(file_path)

In [79]:
def get_class_weights(folder,digits = 1): 
    # determines class weights based on distribution of images in folder --> fewer img = greater weight
    count = []
    for f in os.listdir(folder):
        count.append(len(os.listdir(os.path.join(folder,f))))
    max_c = max(count)
    return [round((max_c / i),digits) for i in count]

weights = get_class_weights(file_path)

In [80]:
data = tf.keras.utils.image_dataset_from_directory(file_path,batch_size=16)
# quick way to get all the images and classes collated

Found 541 files belonging to 8 classes.


In [81]:
data.class_names 
# names of the classes (dog breeds) --> same order as that of class weights

['beagle',
 'bulldog',
 'dalmatian',
 'german-shepherd',
 'husky',
 'labrador-retriever',
 'poodle',
 'rottweiler']

In [82]:
train_p,val_p,test_p = 0.65,0.15,0.2
# split into 65% training, 15% validation, 20% test (0.7/0.1/0/2) also works etc
assert round(train_p + val_p + test_p, 5) == 1 # checks that fractions == 1

# data partitioning process
train_size = int(len(data)*train_p)
val_size = int(len(data)*val_p)
test_size = int(len(data)*test_p)

train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [83]:
# data (img) normalisation
train = train.map(lambda x,y:(x/255,y))
val = val.map(lambda x,y:(x/255,y))
test = test.map(lambda x,y:(x/255,y))

In [84]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomTranslation(0.1, 0.1),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomFlip('horizontal'),
])

# augmentation of training set images --> some being rotated/ translated/ skewed/ flipped
def apply_augmentations(images, labels):
    images = data_augmentation(images)
    return images, labels

train = train.map(apply_augmentations)

In [85]:
# predefined functions to get precision, recall and f1 scores (
# problems with tf metrics --> thus functions below used)

def precision(y_true, y_pred):
    # Convert predictions to class labels
    y_pred = tf.argmax(y_pred, axis=1)
    y_true = tf.cast(y_true, tf.int64)
    y_pred = tf.cast(y_pred, tf.int64)    
    # Compute true positives
    true_positives = tf.reduce_sum(tf.cast(tf.equal(y_true, y_pred), 
        tf.float32))
    
    # Compute false positives
    all_positives = tf.reduce_sum(tf.cast(tf.not_equal(y_pred, 0), 
        tf.float32))  # Assuming background class is labeled as 0
    predicted_positives = tf.reduce_sum(tf.cast(tf.not_equal(y_true, 0), 
        tf.float32))  # Assuming background class is labeled as 0
    false_positives = all_positives - true_positives
    
    # Avoid division by zero
    precision = true_positives / (true_positives + false_positives + tf.keras.backend.epsilon())
    
    return precision

def recall(y_true, y_pred):
    # Convert predictions to class labels
    y_pred = tf.argmax(y_pred, axis=1)
    
    # Cast data types to align them
    y_true = tf.cast(y_true, tf.int64)
    y_pred = tf.cast(y_pred, tf.int64)
    
    # Compute true positives
    true_positives = tf.reduce_sum(tf.cast(tf.equal(y_true, y_pred), tf.float32))
    
    # Compute false negatives
    all_positives = tf.reduce_sum(tf.cast(tf.not_equal(y_pred, 0), 
        tf.float32))  # Assuming background class is labeled as 0
    actual_positives = tf.reduce_sum(tf.cast(tf.not_equal(y_true, 0), 
        tf.float32))  # Assuming background class is labeled as 0
    false_negatives = actual_positives - true_positives
    
    # Avoid division by zero
    recall = true_positives / (true_positives + false_negatives + tf.keras.backend.epsilon())
    
    return recall

def f1_score(y_true, y_pred):
    # Compute precision and recall
    precision_val = precision(y_true, y_pred)
    recall_val = recall(y_true, y_pred)
    
    # Compute F1 score
    f1 = 2 * (precision_val * recall_val) / (precision_val + recall_val + tf.keras.backend.epsilon())
    f1 = tf.clip_by_value(f1, 0.0, 1.0) # clip it to the specified range
    return f1

In [86]:
# Neural Network Model Created

model = Sequential() 
model.add(Input((256,256,3))) # input layer accepting images of 256 x 256 (3 channels -> RGB)
# RGB preferred instead of grayscale since color infomration of dogs can be helpful in distinguishing
# between dogs that have similar features (rounded ears/ pointed snouts) whereby colour can be helpful
# to tell them apart

# Convolution and Pooling Layers Added
model.add(Conv2D(32,(3,3),1,activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(32,(3,3),1,activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16,(3,3),1,activation='relu'))
model.add(MaxPooling2D())

# Layers flattened
model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dense(len(data.class_names),activation='softmax')) # Predictions made

model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy',f1_score])

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(os.curdir,'dog_breeds_callback.keras'),
    monitor = 'val_accuracy',save_best_only=True,verbose=1)

if os.path.exists(os.path.join(os.curdir,'dog_breeds_model.keras')): # if model has already been saved; will not fit another
    pass 
else:
    model.fit(train,validation_data = val,epochs = 50, 
    class_weight={k:v for k,v in enumerate(weights)},callbacks = checkpoint_callback)

Epoch 1/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 722ms/step - accuracy: 0.1231 - f1_score: 0.1414 - loss: 2.5961
Epoch 1: val_accuracy improved from -inf to 0.25000, saving model to .\dog_breeds_callback.keras
22/22 ━━━━━━━━━━━━━━━━━━━━ 23s 863ms/step - accuracy: 0.1241 - f1_score: 0.1423 - loss: 2.5953 - val_accuracy: 0.2500 - val_f1_score: 0.2638 - val_loss: 2.0456
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 780ms/step - accuracy: 0.1668 - f1_score: 0.2016 - loss: 2.4922
Epoch 2: val_accuracy did not improve from 0.25000
22/22 ━━━━━━━━━━━━━━━━━━━━ 20s 880ms/step - accuracy: 0.1671 - f1_score: 0.2020 - loss: 2.4955 - val_accuracy: 0.2375 - val_f1_score: 0.2559 - val_loss: 1.9878
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 719ms/step - accuracy: 0.2261 - f1_score: 0.2439 - loss: 2.5061
Epoch 3: val_accuracy did not improve from 0.25000
22/22 ━━━━━━━━━━━━━━━━━━━━ 19s 822ms/step - accuracy: 0.2266 - f1_score: 0.2442 - loss: 2.5063 - val_accuracy: 0.1750 - val_f1_score: 0.1908 - val_loss: 1.9695
Epoch 

In [90]:
images = []
labels = []

# Iterate through the dataset and collect images and labels
for image_batch, label_batch in test:
    for image, label in zip(image_batch, label_batch):
        images.append(image.numpy())  # Convert to numpy array if needed
        labels.append(label.numpy())  # Convert to numpy array if needed

# Converts lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

In [91]:
if os.path.exists(os.path.join(os.curdir,'dog_breeds_model.keras')): 
    # if model has been saved, pre-saved model loaded
    model = tf.keras.models.load_model(os.path.join(os.curdir,'dog_breeds_model.keras'))
else:
    model.load_weights(os.path.join(os.curdir,'dog_breeds_callback.keras')) 
    # else optimal weights used to create model
    model.save(os.path.join(os.curdir,'dog_breeds_model.keras')) 
    # model is then saved

test_y_pred =  np.argmax(model.predict(images), axis=-1) 
# predictions on test data are formed & evaluated using classification report
print(classification_report(labels,test_y_pred,target_names=data.class_names)) 


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 161ms/step
                    precision    recall  f1-score   support

            beagle       0.87      0.93      0.90        14
           bulldog       0.85      0.58      0.69        19
         dalmatian       0.79      1.00      0.88        11
   german-shepherd       0.77      0.83      0.80        12
             husky       0.62      0.71      0.67         7
labrador-retriever       0.69      0.82      0.75        11
            poodle       1.00      0.83      0.91        12
        rottweiler       0.90      0.90      0.90        10

          accuracy                           0.81        96
         macro avg       0.81      0.83      0.81        96
      weighted avg       0.82      0.81      0.81        96



In [92]:
model.evaluate(test) # compared with tensorflow's metrics which align q well

6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 141ms/step - accuracy: 0.7045 - f1_score: 0.8470 - loss: 0.8844


[0.9115530848503113, 0.7291666865348816, 0.8710581660270691]